````{panels}
Voraussetzungen
^^^
- Statische Modellierung einer Highline ({ref}`highline_static`)
- mathematische Grundlagen von Eigenwerten und Eigenvektoren
---

Lerninhalte
^^^
- Berechnung von Schwingungsmoden mithilfe der Steifigkeits- und Massematrix
````

(beam_eigen_modes)=
# Schwingungsmoden eines eindimensionalen Balkens

Wir wollen einen flexiblen Balken simulieren. Dazu diskretisieren wir ihn durch $N$ Massepunkte der Masse $m$ und verbinden diese mit Federn der Steifigkeit $k$. 

Wir vernachlässigen zur Vereinfachung Torsionssteifigkeiten und betrachten das System bei Schwerelosigkeit.

```{image} images/rod.png
:align: center
```

## Die Modellgleichungen

Das Bild kommt uns bekannt vor! Wir können dieselben Modellgleichungen wie in dem Kapitel {ref}`highline_static` verwenden, allerdings vernachlässigen wir die externe Kraft und die Dämpfung.

Gehen Sie von einem *"Urzustand"* aus, in dem alle Massepunkte auf einer Linie liegen. Durch die Schwerkraft und das Einwirken äußerer Kräfte entstehen Verschiebungen 


$$ \mathbf{u}_i = \begin{bmatrix} x_i(t) \\ y_i(t) \end{bmatrix} $$ 


der Massepunkte weg von diesem Urzustand. Die Kräftebilanzierung ergibt


$$
\begin{align}
 m \ddot{\mathbf{u}}_1 &= -k(\mathbf{u}_1-\mathbf{u}_2)  \notag \\
 m \ddot{\mathbf{u}}_i &= k(\mathbf{u}_{i-1}-\mathbf{u}_{i}) - k(\mathbf{u}_{i}-\mathbf{u}_{i+1}) , \hskip4em \text{für } i=2,...,N-1 \notag \\
 m \ddot{\mathbf{u}}_N &= k(\mathbf{u}_{N-1}-\mathbf{u}_N) . \notag \\
\end{align}
$$


Diese Gleichungen lassen sich in eine kompaktere Matrixschreibweise bringen. Dazu führen wir folgende Bezeichungen ein:


$$ \mathbf{u}=\left[ \begin{array}{c} \mathbf{u}_1 \\ \vdots \\ \mathbf{u}_N \end{array} \right] \in \mathbb{R}^{2N},, \quad M = \left[ \begin{array}{cccc} m & 0 & \cdots & 0 \\ 0 & m & \ddots & \vdots \\ \vdots & \ddots & \ddots & 0 \\ 0 & \cdots & 0 & m \end{array} \right] \in \mathbb{R}^{2N \times 2N} \quad \text{und} $$                                                                                                                                                                                                  

$$S = \left[ \begin{array}{rrrrrrr}
1 & 0 & -1 & 0 & \cdots & \cdots & 0  \\
0 & 1 & 0 & -1 & \ddots &  & \vdots \\
-1 & 0 & 2 & 0 & \ddots & \ddots & \vdots \\
0 & -1 & 0 & \ddots & \ddots & \ddots & 0 \\
\vdots & \ddots & \ddots & \ddots  & 2 & \ddots & -1 \\
\vdots & & \ddots & \ddots  & \ddots & 1 & 0 \\
0 & \cdots & \cdots & 0 & -1 & 0 & 1
\end{array}  \right] \in \mathbb{R}^{2N \times 2N}.$$


$M$ ist die *Massematrix* und $K = k \cdot S$ die *Steifigkeitsmatrix* des gekoppelten Systems. Zusammengesetzt ergibt sich folgende Differentialgleichung:

$$M \ddot{\mathbf{z}} + K\mathbf{z} = \boldsymbol{0} .$$


## Was hat das mit Eigenwerten und Eigenvektoren zu tun?

Wir erkennen die harmonische Schwingungsgleichung und machen einen entsprechenden Ansatz für die Lösung $\mathbf{u}$:


$$
\mathbf{u}(t) = \boldsymbol{\phi} \cdot e^{i \omega t},
$$


wobei $\boldsymbol{\phi} \in \mathbb{R}^{2N}$ die *zeitunabhängige* Amplitude der beiden Komponenten jedes einzelnen Massepunktes beschreibt. Durch Einsetzen unseres Ansatzes in die Differentialgleichung erhalten wir 


$$
M^{-1}K \boldsymbol{\phi} = \omega^2 \boldsymbol{\phi}.
$$


$\boldsymbol{\phi}$ ist also ein **Eigenvektor** von $M^{-1}K$ zum **Eigenwert** $\omega^2$.


 - Es gibt $n$ Paare $(\omega_j, \boldsymbol{\phi}_j)$, wobei $n=\text{rang}(M^{-1}K)$
 - $\omega_j$ nennt man **Eigenfrequenz** des Systems
 - $\boldsymbol{\phi}_j$ nennt man **Schwingungsmode** oder **Schwingungsform** des Systems.
 
 
Das System hat also viele Eigenfrequenzen. Wenn wir es mit einer bestimmten Eigenfrequenz anregen, stellt sich eine bestimmte Schwingungsform ein, die im Amplitudenvektor $\boldsymbol{\phi}$ kodiert ist. Jede Bewegung die das System ausführen kann, lässt sich als Superposition der Schwingungsmoden darstellen.


Zur Anschauung einzelner Schwingungsmoden $\boldsymbol{\phi}_j$ animiert man sie oft mit einer festen Frequenz $a \neq \omega_j$ und ohne Phasenverschiebung. Das folgende Skript berechnet die Eigenfrequenzen und Schwingungsmoden, in dem das entsprechende Eigenwertproblem gelöst wird, und animiert die Schwingungsmoden. 

```{admonition} Hinweis
Wegen der Animation kann das Skript nicht interaktiv ausgeführt werden, laden Sie es sich herunter.
```

In [ ]:
clear
saveImages = false;

%%

T =10*pi;  % total animation time [s] per mode
dt=pi/5;   % time step size for the animation [s]

L = 10;    % length of the rod [m]
N = 20;    % number of mass points of the rod
k = 4e6*N; % stiffness of each rod segment [N/m]
m = 10/N;  % mass of each mass point

%% construct stiffness matrix

n = 2*N;           % dimension of the problem is 2D

% construct matrix S as a sparse matrix
nnz = n + 2*(n-2); % number of nonzeros
row    = zeros(nnz,1);  %preallocate memory
column = zeros(nnz,1);
value  = zeros(nnz,1);
j=0;
for i=1:n %for all rows
    if ( i>2 && i<n-1 )
        j=j+1; row(j)=i; column(j)=i; value(j)=2; %diagonal element
    else
        j=j+1; row(j)=i; column(j)=i; value(j)=1; %diagonal element
    end
    if i>2
        j=j+1; row(j)=i; column(j)=i-2; value(j)=-1; % +2 diagonal
    end
    if i<n-1
        j=j+1; row(j)=i; column(j)=i+2; value(j)=-1; % -2 diagonal
    end
end
S = sparse(row,column,value,n,n);

% stiffness matrix (no need for mass matrix)
K = k*S;


%% calculate eigenfrequencies
[V, D] = eig(full(K/m));
eigfreqs = sqrt(diag(D));

%% animate the i--th frequency mode
k=0;
for i=1:12
    
    % get the modal shapes (eigenvectors)
    qi = V(:,i)';
    
    for t=0:dt:T
        
        k=k+1;
        
        %position of mass points = displacement + initial position
        % also scale the x-direction displacements
        positions_x = 2*qi(1:2:n)*sin(t)+L*linspace(1/N,1-1/N,N);
        positions_y = qi(2:2:n)*sin(t);
        
        %% plot the animation of the rod
        cla;
        hold on
        plot(positions_x,positions_y,'k','Marker','o','LineWidth',5,'MarkerSize',15);
        box on
        ylim([-1,1]);
        xlim([-0.1*L,1.1*L]);
        xticks([]);
        yticks([]);
        title(['mode ',num2str(i),', freq = ',num2str(eigfreqs(i)/(2*pi),'%1.2f'),' Hz'],'fontsize',20)
        
        drawnow
        
        if saveImages
            disp(['mode ',num2str(i), ', total image number ',num2str(k)])
            filename = ['rod_modes_',num2str(k,'%04.0f'),'.png'];
            saveas(gcf,filename)
        end
    end
end

```{image} images/modes.gif
:align: center
```

Die ersten beiden Schwingungsmoden zum Eigenwert 0 *(genauer: Singulärewert)* sind die sogenannte **Starrkörpermoden**, bei denen keine Verformung stattfindet.

## Ihre Aufgabe

Modifizieren Sie das Skript so, dass der eindimensionale Balken

### (a) einseitig eingespannt ist.
### (b) beidseitig eingespannt ist.

**Tipp:** Nehmen Sie Änderungen an der Matrix $S$ vor.